In [1]:
## Import Libraries
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import random
import torch
import seaborn as sns
from collections import Counter
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential,layers, losses, optimizers
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from keras.layers import GaussianNoise
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score
from keras.layers import LeakyReLU

In [2]:
## Loading adversarial training data
df_train = pd.read_csv("Adv_all_DNN_incremental.csv")

In [3]:
df_train

,Unnamed: 0,ip_header_byte_2,ip_header_byte_3,ip_header_byte_4,ip_header_byte_5,ip_header_byte_6,ip_header_byte_7,ip_header_byte_8,ip_header_byte_10,ip_header_byte_11,...,tcp_segment_data_byte_1450,tcp_segment_data_byte_1451,tcp_segment_data_byte_1452,tcp_segment_data_byte_1453,tcp_segment_data_byte_1454,tcp_segment_data_byte_1455,tcp_segment_data_byte_1456,tcp_segment_data_byte_1457,tcp_segment_data_byte_1458,tcp_segment_data_byte_1459
0,0,0.007843,0.686275,0.219608,0.941176,0.25098,0.0,0.501961,0.505882,0.690196,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.000000,0.235294,0.113725,0.478431,0.25098,0.0,0.243137,0.658824,0.337255,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.000000,0.235294,0.682353,0.352941,0.25098,0.0,0.243137,0.090196,0.462745,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.007843,0.109804,0.407843,0.898039,0.25098,0.0,0.501961,0.321569,0.305882,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.011765,0.086275,0.145098,0.552941,0.25098,0.0,0.501961,0.580392,0.674510,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5180,5180,0.007843,0.490196,0.658824,0.286275,0.25098,0.0,0.243137,0.105882,0.274510,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5181,5181,0.007843,0.490196,0.866667,0.670588,0.25098,0.0,0.243137,0.898039,0.890196,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5182,5182,0.003922,0.701961,0.380392,0.160784,0.25098,0.0,0.243137,0.388235,0.188235,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5183,5183,0.007843,0.682353,0.035294,0.796078,0.25098,0.0,0.501961,0.690196,0.839216,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
df_train = df_train.loc[:, ~df_train.columns.str.contains('^Unnamed')]

In [5]:
df_train

,ip_header_byte_2,ip_header_byte_3,ip_header_byte_4,ip_header_byte_5,ip_header_byte_6,ip_header_byte_7,ip_header_byte_8,ip_header_byte_10,ip_header_byte_11,tcp_header_byte_4,...,tcp_segment_data_byte_1450,tcp_segment_data_byte_1451,tcp_segment_data_byte_1452,tcp_segment_data_byte_1453,tcp_segment_data_byte_1454,tcp_segment_data_byte_1455,tcp_segment_data_byte_1456,tcp_segment_data_byte_1457,tcp_segment_data_byte_1458,tcp_segment_data_byte_1459
0,0.007843,0.686275,0.219608,0.941176,0.25098,0.0,0.501961,0.505882,0.690196,0.196078,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.235294,0.113725,0.478431,0.25098,0.0,0.243137,0.658824,0.337255,0.392157,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.235294,0.682353,0.352941,0.25098,0.0,0.243137,0.090196,0.462745,0.039216,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.007843,0.109804,0.407843,0.898039,0.25098,0.0,0.501961,0.321569,0.305882,0.192157,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.011765,0.086275,0.145098,0.552941,0.25098,0.0,0.501961,0.580392,0.674510,0.196078,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5180,0.007843,0.490196,0.658824,0.286275,0.25098,0.0,0.243137,0.105882,0.274510,0.172549,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5181,0.007843,0.490196,0.866667,0.670588,0.25098,0.0,0.243137,0.898039,0.890196,0.105882,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5182,0.003922,0.701961,0.380392,0.160784,0.25098,0.0,0.243137,0.388235,0.188235,0.611765,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5183,0.007843,0.682353,0.035294,0.796078,0.25098,0.0,0.501961,0.690196,0.839216,0.929412,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
adv_data = df_train.iloc[:,0:500]

In [7]:
adv_data

,ip_header_byte_2,ip_header_byte_3,ip_header_byte_4,ip_header_byte_5,ip_header_byte_6,ip_header_byte_7,ip_header_byte_8,ip_header_byte_10,ip_header_byte_11,tcp_header_byte_4,...,tcp_segment_data_byte_425,tcp_segment_data_byte_426,tcp_segment_data_byte_427,tcp_segment_data_byte_428,tcp_segment_data_byte_429,tcp_segment_data_byte_430,tcp_segment_data_byte_431,tcp_segment_data_byte_432,tcp_segment_data_byte_433,tcp_segment_data_byte_434
0,0.007843,0.686275,0.219608,0.941176,0.25098,0.0,0.501961,0.505882,0.690196,0.196078,...,0.450980,0.125490,0.380392,0.431373,0.125490,0.411765,0.431373,0.454902,0.396078,0.447059
1,0.000000,0.235294,0.113725,0.478431,0.25098,0.0,0.243137,0.658824,0.337255,0.392157,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.235294,0.682353,0.352941,0.25098,0.0,0.243137,0.090196,0.462745,0.039216,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.007843,0.109804,0.407843,0.898039,0.25098,0.0,0.501961,0.321569,0.305882,0.192157,...,0.423529,0.125490,0.388235,0.435294,0.427451,0.427451,0.380392,0.431373,0.392157,0.172549
4,0.011765,0.086275,0.145098,0.552941,0.25098,0.0,0.501961,0.580392,0.674510,0.196078,...,0.423529,0.125490,0.388235,0.435294,0.427451,0.427451,0.380392,0.431373,0.392157,0.172549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5180,0.007843,0.490196,0.658824,0.286275,0.25098,0.0,0.243137,0.105882,0.274510,0.172549,...,0.050980,0.039216,0.321569,0.396078,0.400000,0.396078,0.447059,0.396078,0.447059,0.227451
5181,0.007843,0.490196,0.866667,0.670588,0.25098,0.0,0.243137,0.898039,0.890196,0.105882,...,0.050980,0.039216,0.321569,0.396078,0.400000,0.396078,0.447059,0.396078,0.447059,0.227451
5182,0.003922,0.701961,0.380392,0.160784,0.25098,0.0,0.243137,0.388235,0.188235,0.611765,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5183,0.007843,0.682353,0.035294,0.796078,0.25098,0.0,0.501961,0.690196,0.839216,0.929412,...,0.125490,0.380392,0.431373,0.125490,0.411765,0.431373,0.454902,0.396078,0.447059,0.431373


In [8]:
## AE model architecture
tf.random.set_seed(2)
input_dims = 500
model = Sequential()
model.add(Dense(input_dims, input_shape = (input_dims, )))
model.add(Dense(400))
model.add(LeakyReLU(alpha=0.05))
model.add(Dense(300))
model.add(LeakyReLU(alpha=0.05))
model.add(Dense(input_dims // 2, activation='relu'))
model.add(Dense(input_dims // 4, activation='relu'))
model.add(Dense(input_dims // 8, activation='relu'))
model.add(Dense(input_dims // 4, activation='relu'))
model.add(Dense(input_dims // 2))
model.add(Dense(300))
model.add(LeakyReLU(alpha=0.05))
model.add(Dense(400))
model.add(LeakyReLU(alpha=0.05))
model.add(Dense(input_dims, activation='relu'))

In [1]:
## Compilation and training
model.compile(optimizer='adam', loss='mae')
model.fit(adv_data , adv_data , batch_size=256, epochs=500, shuffle=True)

In [52]:
## Saving the stage-3 base line model
model.save('Stage3_AE.h5')

In [11]:
## Loading the stage-3 adversarial model
from tensorflow.keras.models import load_model

model = load_model('Stage3_AE.h5', compile = False)

In [12]:
## pbRe metric
def recon_metrices(data, reconstructed_data):

    maes = np.absolute(data.values - reconstructed_data)
    
    for j in range(len(maes)):
        m = maes[j]
        d = data.values[j]
        r = reconstructed_data[j]
        for el in range(len(m)):
            if r[el] == 0 or d[el] == 0:
                m[el] = 0
 
            
    mae_byte_list = []
    ip_mae_list = []
    tcp_header_mae_list = []
    tcp_options_mae_list = []
    tcp_segment_mae_list = []
    i=0
    for mae in maes:
#         print(np.count_nonzero(reconstructed_data[i]))
#         print(np.count_nonzero(data.values[i]))
#         print(np.max(np.count_nonzero(reconstructed_data[i]),np.count_nonzero(data.values[i])))
        mae_byte = sum(mae) / np.count_nonzero(mae)
        mae_byte_list.append(mae_byte)
        i += 1
    return mae_byte_list

In [ ]:
## Constructing threshold on test adversarial data

In [13]:
recon_adv = model.predict(adv_data)

In [14]:
mae_byte_list = recon_metrices(adv_data,recon_adv)

In [16]:
## Thresholds for 99,95 and 90 percentile values
mae_byte_avg = sum(mae_byte_list) / len(mae_byte_list)
print('mae_byte_avg', mae_byte_avg)
mae_byte_per_95 = np.percentile(mae_byte_list, 99)
print('mae_byte_99_percentile', mae_byte_per_95)
mae_byte_per_95 = np.percentile(mae_byte_list, 95)
print('mae_byte_95_percentile', mae_byte_per_95)
mae_byte_per_95 = np.percentile(mae_byte_list, 90)
print('mae_byte_90_percentile', mae_byte_per_95)

mae_byte_avg 0.02721943360424446
mae_byte_99_percentile 0.16476026957077683
mae_byte_95_percentile 0.105800987740455
mae_byte_90_percentile 0.060493268813147684


In [44]:
## Loading the test dataset
df_test = pd.read_csv("Adv_all_DNN_truerandom.csv")

In [45]:
df_test = df_test.loc[:, ~df_test.columns.str.contains('^Unnamed')]

In [46]:
df_test

,ip_header_byte_2,ip_header_byte_3,ip_header_byte_4,ip_header_byte_5,ip_header_byte_6,ip_header_byte_7,ip_header_byte_8,ip_header_byte_10,ip_header_byte_11,tcp_header_byte_4,...,tcp_segment_data_byte_1450,tcp_segment_data_byte_1451,tcp_segment_data_byte_1452,tcp_segment_data_byte_1453,tcp_segment_data_byte_1454,tcp_segment_data_byte_1455,tcp_segment_data_byte_1456,tcp_segment_data_byte_1457,tcp_segment_data_byte_1458,tcp_segment_data_byte_1459
0,0.007843,0.686275,0.219608,0.941176,0.25098,0.0,0.501961,0.505882,0.690196,0.196078,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.235294,0.682353,0.352941,0.25098,0.0,0.243137,0.090196,0.462745,0.039216,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.007843,0.109804,0.407843,0.898039,0.25098,0.0,0.501961,0.321569,0.305882,0.192157,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.011765,0.086275,0.145098,0.552941,0.25098,0.0,0.501961,0.580392,0.674510,0.196078,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.003922,0.701961,0.894118,0.317647,0.25098,0.0,0.243137,0.878431,0.027451,0.192157,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9402,0.015686,0.058824,0.498039,0.545098,0.25098,0.0,0.501961,0.223529,0.709804,0.196078,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9403,0.003922,0.623529,0.427451,0.545098,0.25098,0.0,0.501961,0.305882,0.145098,0.196078,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9404,0.000000,0.203922,0.709804,0.580392,0.25098,0.0,0.243137,0.062745,0.266667,0.400000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9405,0.007843,0.490196,0.239216,0.043137,0.25098,0.0,0.243137,0.525490,0.517647,0.200000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
## Testing the baseline model against different adversarial samples and CICIDS2018 data 
##that leaked from the previous stages

In [47]:
adv_data_test = df_test.iloc[:, 0:500]

In [48]:
recon_adv_test = model.predict(adv_data_test)

294/294 [==============================] - 1s 5ms/step


In [49]:
mae_byte_list_adv = recon_metrices(adv_data_test,recon_adv_test)

In [50]:
mae_byte_avg = sum(mae_byte_list_adv) / len(mae_byte_list_adv)
print('mae_byte_avg', mae_byte_avg)
mae_byte_per_95 = np.percentile(mae_byte_list_adv, 99)
print('mae_byte_99_percentile', mae_byte_per_95)
mae_byte_per_95 = np.percentile(mae_byte_list_adv, 95)
print('mae_byte_95_percentile', mae_byte_per_95)
mae_byte_per_95 = np.percentile(mae_byte_list_adv, 90)
print('mae_byte_90_percentile', mae_byte_per_95)

mae_byte_avg 0.06995723034191342
mae_byte_99_percentile 0.2702800883978238
mae_byte_95_percentile 0.21976715405544195
mae_byte_90_percentile 0.176788629950701


In [51]:
## Classification
count_99 = len([i for i in mae_byte_list_adv if i > 0.09])
print(count_99 / len(mae_byte_list_adv))

0.30636759859678964


In [52]:
df_test

,ip_header_byte_2,ip_header_byte_3,ip_header_byte_4,ip_header_byte_5,ip_header_byte_6,ip_header_byte_7,ip_header_byte_8,ip_header_byte_10,ip_header_byte_11,tcp_header_byte_4,...,tcp_segment_data_byte_1450,tcp_segment_data_byte_1451,tcp_segment_data_byte_1452,tcp_segment_data_byte_1453,tcp_segment_data_byte_1454,tcp_segment_data_byte_1455,tcp_segment_data_byte_1456,tcp_segment_data_byte_1457,tcp_segment_data_byte_1458,tcp_segment_data_byte_1459
0,0.007843,0.686275,0.219608,0.941176,0.25098,0.0,0.501961,0.505882,0.690196,0.196078,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.235294,0.682353,0.352941,0.25098,0.0,0.243137,0.090196,0.462745,0.039216,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.007843,0.109804,0.407843,0.898039,0.25098,0.0,0.501961,0.321569,0.305882,0.192157,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.011765,0.086275,0.145098,0.552941,0.25098,0.0,0.501961,0.580392,0.674510,0.196078,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.003922,0.701961,0.894118,0.317647,0.25098,0.0,0.243137,0.878431,0.027451,0.192157,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9402,0.015686,0.058824,0.498039,0.545098,0.25098,0.0,0.501961,0.223529,0.709804,0.196078,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9403,0.003922,0.623529,0.427451,0.545098,0.25098,0.0,0.501961,0.305882,0.145098,0.196078,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9404,0.000000,0.203922,0.709804,0.580392,0.25098,0.0,0.243137,0.062745,0.266667,0.400000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9405,0.007843,0.490196,0.239216,0.043137,0.25098,0.0,0.243137,0.525490,0.517647,0.200000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
adv_all = pd.read_csv("adv_examples_all_allDNN_bigger.csv")

In [54]:
adv_data_RL = adv_all.iloc[:,0:500]

In [55]:
recon_adv_RL = model.predict(adv_data_RL)

362/362 [==============================] - 2s 6ms/step


In [56]:
mae_byte_list_adv_RL = recon_metrices(adv_data_RL,recon_adv_RL)

In [57]:
mae_byte_avg = sum(mae_byte_list_adv_RL) / len(mae_byte_list_adv_RL)
print('mae_byte_avg', mae_byte_avg)
mae_byte_per_95 = np.percentile(mae_byte_list_adv_RL, 99)
print('mae_byte_99_percentile', mae_byte_per_95)
mae_byte_per_95 = np.percentile(mae_byte_list_adv_RL, 95)
print('mae_byte_95_percentile', mae_byte_per_95)
mae_byte_per_95 = np.percentile(mae_byte_list_adv_RL, 90)
print('mae_byte_90_percentile', mae_byte_per_95)

mae_byte_avg 0.11438805513581005
mae_byte_99_percentile 0.20714375468247465
mae_byte_95_percentile 0.171608887575364
mae_byte_90_percentile 0.157526317870304


In [58]:
## Classification
count_99 = len([i for i in mae_byte_list_adv_RL if i > 0.09])
print(count_99 / len(mae_byte_list_adv_RL))

0.7298910223144784


In [59]:
df_2018 = pd.read_csv("all_group_train_normalized_2018.csv")

In [60]:
infiltration_attack = df_2018[df_2018['Label'] == 'Infiltration'].drop(['Label_binary','Label'], axis=1).iloc[:,0:500]
print(infiltration_attack.shape)
# portscan_attack = df_2018[df_2018['Label'] == 'Port Scan'].drop(['Label_binary','Label'], axis=1).iloc[:,0:500]
# print(portscan_attack.shape)
ddos_attack = df_2018[df_2018['Label'] == 'ddos'].drop(['Label_binary','Label'], axis=1).iloc[:,0:500]
print(ddos_attack.shape)
bruteforce_attack = df_2018[df_2018['Label'] == 'Brute Force'].drop(['Label_binary','Label'], axis=1).iloc[:,0:500]
print(bruteforce_attack.shape)
dos_attack = df_2018[df_2018['Label'] == 'DoS'].drop(['Label_binary','Label'], axis=1).iloc[:,0:500]
print(dos_attack.shape)
web_attack = df_2018[df_2018['Label'] == 'Web Attack'].drop(['Label_binary','Label'], axis=1).iloc[:,0:500]
print(web_attack.shape)
benign_data = df_2018[df_2018['Label'] == 'Benign'].drop(['Label_binary','Label'], axis=1).iloc[:,0:500]
print(benign_data.shape)

(29116, 500)
(29115, 500)
(29116, 500)
(29116, 500)
(29116, 500)
(145579, 500)


In [61]:
recon_inf = model.predict(infiltration_attack)
recon_dos = model.predict(dos_attack)
recon_ddos = model.predict(ddos_attack)
recon_web = model.predict(web_attack)
recon_brute = model.predict(bruteforce_attack)
recon_ben = model.predict(benign_data)

4550/4550 [==============================] - 39s 8ms/step


In [62]:
mae_byte_list = recon_metrices(benign_data,recon_ben)

In [63]:
## Classification
count_99 = len([i for i in mae_byte_list if i > 0.09])
print(count_99 / len(mae_byte_list))

0.8555148750850053
